In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4,ErrorCode, plotTrajectoriesFile,Variable
from datetime import timedelta, datetime
import numpy as np
import xarray
from operator import attrgetter
import math

In [2]:
#We can add or remove all the zeros according to preference. In case that they are left there, we only get daily data for the currents which will end up with the code running faster, but we do lose time resolution. Tests will determine if this loss in time resolution is actually important
filenames = {'U': "/Volumes/4YP/Data/Total_24hr_new/Total*.nc",
             'V': "/Volumes/4YP/Data/Total_24hr_new/Total*.nc",
             'borU':"/Volumes/4YP/Data/Boundary_Velocity_24hr/BV*.nc",
             'borV':"/Volumes/4YP/Data/Boundary_Velocity_24hr/BV*.nc"}
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity',
             'borU':'MaskUvel',
             'borV':'MaskVvel'}
dimensions = {'U':{'lat': 'latitude','lon': 'longitude','time': 'time'},
              'V':{'lat': 'latitude','lon': 'longitude','time': 'time'},
              'borU':{'lat': 'lat','lon': 'lon','time': 'time'},
              'borV':{'lat': 'lat','lon': 'lon','time': 'time'},
             }
              

In [3]:
#Create the fieldset with the periodic halo and time extrapolation for the EKE
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation=True)
fieldset.add_periodic_halo(zonal=True)

In [4]:
#The starting coordinates of the Particles, for the North Pacific. They are generated
#by the code NAgrid.py, graciously send to me by David.
lons=np.load('/Volumes/4YP/Data/Particles_lon.npy')
lats=np.load('/Volumes/4YP/Data/Particles_lat.npy')
#lons, lats = np.meshgrid(lon,lat)
lons[lons>180]-=360

In [5]:
#And now we define what sort of particles we are actually dealing with
class SampleParticle(JITParticle):
#    #Now the part to determine the age of the particle
    Age=Variable('Age',initial=0.,dtype=np.float32)#agr is gonna be in seconds
    prev_time=Variable('prev_time',initial=attrgetter('time'),to_write=False)
    #Now the part to track the distance covered
#    distance = Variable('distance', initial=0., dtype=np.float32)
#    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lon'))
#    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lat'))
#    #Now I also want the particle to be deleted if it is on land (so it won't move)
#    count=Variable('count',initial=0,to_write=False)
#    init_lon = Variable('init_lon', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lon'))
#    init_lat = Variable('init_lat', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lat'))

In [6]:
#The starting point of the similation and the endtime
starttime=datetime(2002,1,1,0,0)
endtime=datetime(2014,12,31,21,0)
pset = ParticleSet(fieldset=fieldset, pclass=SampleParticle, lon=lons, lat=lats,time=starttime)

In [7]:
#%% All the different functions/kernels we want to have
def DeleteParticle(particle, fieldset, time, dt):
    particle.delete()
    print('we deleted it at '+str(particle.lon)+' and '+str(particle.lat))
def AgeSample(particle, fiedset,time,dt):
    current_time=particle.time
    timedifference=current_time-particle.prev_time
    particle.Age+=timedifference
    particle.prev_time=current_time
def antiBeach(particle,fieldset,time,dt):
    bu=fieldset.borU[time,particle.lon,particle.lat,particle.depth]
    bv=fieldset.borV[time,particle.lon,particle.lat,particle.depth]
    particle.lon-=bu*dt*0.00001
    particle.lat-=bv*dt*0.00001
Agesam=pset.Kernel(AgeSample)    
Beach=pset.Kernel(antiBeach)
#Landsam=pset.Kernel(antiInitialLand)
totalKernal=AdvectionRK4+Beach+Agesam

In [ ]:
pfile = pset.ParticleFile(name="/Volumes/4YP/Data/Trajectories/Global_Total_Test",
                          outputdt=timedelta(hours=48))

Time=starttime
steps=0
while Time<=endtime:
    steps+=1
    Time+=timedelta(hours=48)

for i in range(steps-1):
    pset.execute(totalKernal,
                 runtime=timedelta(hours=48),  # runtime controls the interval of the plots
                 dt=timedelta(minutes=30),
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
                 output_file=pfile
                 )  # the recovery kernel

INFO: Compiled SampleParticleAdvectionRK4antiBeachAgeSample ==> /var/folders/kh/l0nftmqx1wsgmy0j289n8dgc0000gn/T/parcels-501/722482c8f355e7a0754816a051a49d1a.so


In [12]:
file = xarray.open_dataset("/Volumes/4YP/Data/Trajectories/Global_Total_2006.nc")

In [13]:
print(file)

<xarray.Dataset>
Dimensions:     (obs: 2374, traj: 35145)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) int32 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float32 ...
    lon         (traj, obs) float32 ...
    z           (traj, obs) float32 ...
    Age         (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        1.1.1
    parcels_mesh:           spherical


In [16]:
file = xarray.open_dataset("/Volumes/4YP/Data/Trajectories/Global_Total.nc")

In [21]:
print(file.lat.values)

[[-69.125    -68.62582  -68.0592   ... -29.394514 -29.310421 -29.32421 ]
 [-68.125    -67.850426 -67.73982  ... -28.496605 -28.458593 -28.529087]
 [-67.125    -66.91757  -66.83777  ... -29.697302 -29.546995 -29.5039  ]
 ...
 [ 60.875     60.520523  60.313282 ...  36.389713  36.499386  36.661167]
 [ 61.875     61.468437  61.289753 ...  50.624947  50.624947  50.624947]
 [ 63.875     63.672264  63.6211   ...  58.937492  58.937492  58.937492]]
